In [ ]:
from snowflake.snowpark import Session

# Assuming the session is already created and authenticated
# If not, create the session using your credentials

# Set the role to ACCOUNTADMIN
session.sql("USE ROLE ACCOUNTADMIN;").collect()

# Create the table RAW_CO2.Daily_Measurements
create_table_query = """
CREATE OR REPLACE TABLE RAW_CO2.Daily_Measurements (
    date STRING,
    co2_ppm FLOAT
);
"""
session.sql(create_table_query).collect()

# Grant privileges on the table to the ACCOUNTADMIN role
grant_privileges_query = """
GRANT ALL PRIVILEGES ON TABLE RAW_CO2.Daily_Measurements TO ROLE ACCOUNTADMIN;
"""
session.sql(grant_privileges_query).collect()

# Copy data from the stage into the table
copy_into_table_query = """
COPY INTO RAW_CO2.Daily_Measurements
    FROM @RAW_CO2.CO2_EXTERNAL_STAGE
    FILE_FORMAT = (
        TYPE = 'CSV' 
        SKIP_HEADER = 1
        FIELD_OPTIONALLY_ENCLOSED_BY = '"'
    )
    ON_ERROR = 'CONTINUE';
"""
session.sql(copy_into_table_query).collect()

# Create a stream on the table RAW_CO2.DAILY_MEASUREMENTS_STREAM
create_stream_query = """
CREATE OR REPLACE STREAM RAW_CO2.DAILY_MEASUREMENTS_STREAM 
ON TABLE RAW_CO2.Daily_Measurements;
"""
session.sql(create_stream_query).collect()

print("Operations completed successfully.")
